## Part 1 Test 1


In [54]:
import requests
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

headers = {"User-Agent": r'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [81]:
 def write_page(url, file_path): 
    '''
    Opens url and writes html to the file path
    '''
    with open(file_path, 'w',  encoding='utf-8') as file:
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.text, 'lxml')
        file.write(str(soup))

In [94]:
def selenium_activate(key):
    '''
    Selenium gets to a page with a table of 10-k filings. Saves the (cleaned) links with their date. 
    Returns dictionary or URL and Date.
    '''
    driver = webdriver.Chrome()
    print('loading up website')
    
    # Navigates to website
    driver.get('https://www.sec.gov/edgar/searchedgar/companysearch.html')
    #time.sleep(5)
    # CHECK FOR POP UP!!
    
    #Click on the search bar using the full xpath
    xpath_search = r'/html/body/div[2]/div/div/div/section/div[3]/div[2]/div[2]/div[3]/div/form/input[1]'
    driver.find_element_by_xpath(xpath_search).click()
    #time.sleep(2)
    

    #Search bar using full xpath
    searchinput_path = r'/html/body/div[2]/div/div/div/section/div[3]/div[2]/div[2]/div[3]/div/form/input[1]'
    driver.find_element_by_xpath(searchinput_path).send_keys(key, Keys.ENTER)
#     time.sleep(2)
    
    # Click on the 10-K (annual reports) and 10-Q (quarterly reports) tab using full xpath
    w = WebDriverWait(driver,30)
    w.until(expected_conditions.element_to_be_clickable((By.XPATH, r'/html/body/main/div[4]/div[2]/div[3]/h5')))
    button1 = driver.find_element_by_xpath(r'/html/body/main/div[4]/div[2]/div[3]/h5')
    button1.click()
#     time.sleep(2)
    
    # Click on the view all 10-Ks and 10-Qs button using full xpath
    w.until(expected_conditions.element_to_be_clickable((By.XPATH, r'/html/body/main/div[4]/div[2]/div[3]/div/button[1]')))
    button2 = driver.find_element_by_xpath(r'/html/body/main/div[4]/div[2]/div[3]/div/button[1]')
    button2.click()
    #time.sleep(2)
    
    #Click on search table search bar using full xpath
    w.until(expected_conditions.element_to_be_clickable((By.XPATH, r'/html/body/main/div[5]/div/div[1]/div[1]/div/input[2]')))
    xpath_search = r'/html/body/main/div[5]/div/div[1]/div[1]/div/input[2]'
    driver.find_element_by_xpath(xpath_search).click()
    #time.sleep(2)
    

    searchinput_path = r'/html/body/main/div[5]/div/div[1]/div[1]/div/input[2]'
    driver.find_element_by_xpath(searchinput_path).send_keys('10-K', Keys.ENTER)
    
    print('getting urls')
    time.sleep(2)
    
    #Empty url dictionary to put the htmls in
    url = {}
    
    #this bit needs sorting :) tried to make a function but didn't work
    #Rows in this table for some reason are <tr role = "row" class="odd"> or <tr role = "row" class="even">
    #Iterating through each row
    for row in driver.find_elements_by_css_selector("tr.odd"):
        # In each row the annual report link is <a class="document-link"
        #Not sure why this iterates as there's only one per row??
        for cell in row.find_elements_by_class_name("document-link"):
            # The link is an attribute under href= 
            x = cell.get_attribute("href")
            import re
            #re.sub(pattern,repl,string,count=0,flags=0) returns the string obtained by replacing the leftmost non-
            #overlapping occurence of pattern in string by replacement repl
            # This is simply removing the "/ix\?doc=" and saving as x
            x = re.sub(r"/ix\?doc=", '', x)
            time.sleep(1)
            
            #td is each cell in the row, td[3] is the 4th column that hs the report date
        y = row.find_elements_by_tag_name("td")[3].text
        time.sleep(1)
        
        #inputs the html link with the reporting date
        url[x] = y
        
        # Same as above but for every other row
    for row in driver.find_elements_by_css_selector("tr.even"):
        for cell in row.find_elements_by_class_name("document-link"):
            x = cell.get_attribute("href")
            import re
            x = re.sub(r"/ix\?doc=", '', x)
            time.sleep(1)
        y = row.find_elements_by_tag_name("td")[3].text
        time.sleep(1)
        url[x] = y
        
    print('urls gathered, chrome closing')
    #CLOSING THE DRIVER!
    driver.quit()
    return url

In [91]:
def download_files_10k(ticker,dest_folder):
    '''
    Changes directory to destination folder. Calls the selenium to get a link, date dictionary. Uses these to write the html to the file name.
    '''
    os.chdir(dest_folder)
    #Iterate through each company ticker
    for tick in ticker:
        urls= selenium_activate(tick)
        print('Saving html')
        
#         Save each url with the report date and the html version of the link
        for key,value in urls.items(): 
            write_page(key, f'{tick}_10-K_{value}.html')
            time.sleep(1)
    print('complete')

In [95]:
download_files_10k(['AAPL','F','UPS'], r"C:\Users\EmilyThomson\OneDrive - Kubrick Group\Python\EDGAR\Test")

loading up website
getting urls
urls gathered, chrome closing
Saving html
loading up website
getting urls
urls gathered, chrome closing
Saving html
loading up website
getting urls
urls gathered, chrome closing
Saving html
complete
